In [1]:
import pandas as pd 

data = pd.read_csv("broker.csv").replace('-', '', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"], downcast="float")

#data.head()

In [2]:
totalGreens = data['Größe in m² (Kataster)'].sum()
print("%s square metre" % totalGreens)
print("%s hectare" % (totalGreens/10000))

61376304.0 square metre
6137.6304 hectare


In [3]:
#len(data['Nummer des Planungsraumes'].unique().tolist())

In [4]:
#list(filter(None, data['Name Planungsraum'].unique().tolist()))

In [5]:
#data.columns

In [6]:
#data_open_spaces = data.groupby('Nummer des Planungsraumes', as_index=False).agg({"Größe in m² (Kataster)": "sum"})
data_open_spaces = data_open_spaces.drop([0, 411])
data_open_spaces

NameError: name 'data_open_spaces' is not defined

In [ ]:
#data.groupby('Name Planungsraum', as_index=False).size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()


In [ ]:
#data.groupby('Bezirk').size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()

In [ ]:
#empty_PLR = data["Name Planungsraum"] == ""
#data[empty_PLR]

In [ ]:
#import pandas as pd 
df = pd.read_csv('EWR201812E_Matrix.csv', sep=';')

einwohner_gesamt = df['E_E'].sum()

data.rename(columns = {'Nummer des Planungsraumes':'PLR_ID'}, inplace = True) 
df.rename(columns = {'RAUMID':'PLR_ID'}, inplace = True)
data_open_spaces.rename(columns = {'Nummer des Planungsraumes':'PLR_ID'}, inplace = True) 

dataNew = data.groupby(['PLR_ID', 'Name Planungsraum'], as_index=False).size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False)

df = pd.DataFrame(df)
df2 = pd.DataFrame(dataNew)

#print (df2[pd.to_numeric(df2.PLR_ID, errors='coerce').isnull()])
df2 = df2.drop([0, 411])

df2['PLR_ID'] = pd.to_numeric(df2['PLR_ID'])
data_open_spaces['PLR_ID'] = pd.to_numeric(df2['PLR_ID'])

df_merge_col = pd.merge(df, df2, on='PLR_ID')

df_new = pd.merge(df_merge_col, data_open_spaces, on='PLR_ID')

df_new['Grünflächen in h'] = df_new['Größe in m² (Kataster)'].transform(lambda x: x / 10000)

df_new.rename(columns={'Name Planungsraum': 'Planungsraum', 'E_E': 'Einwohner', 'Anzahl der Grünflächen': 'Grünflächen' }, inplace=True)
df_new[['PLR_ID', 'Planungsraum', 'Einwohner', 'Grünflächen', 'Grünflächen in h']]

data_LOR = pd.read_csv("LOR-Planungsraeume.csv")
data_LOR.rename(columns = {'PLR':'PLR_ID'}, inplace = True)

output = pd.merge(data_LOR, df_new, on='PLR_ID')
output.rename(columns={'BEZNAME': 'Bezirk', 'FLAECHE_HA': 'PLR in h'}, inplace=True)

output['PLR in h'] = output['PLR in h'].apply(lambda x: x.replace(',','.'))
output = output[['PLR_ID', 'Planungsraum', 'Einwohner', 'Grünflächen', 'Grünflächen in h', 'Bezirk', 'PLR in h']].sort_values(['PLR_ID'], ascending=True)

output.to_csv('open_spaces_PLR.csv', index = False, header=True)
